In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder,MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

In [2]:
df=pd.read_csv('tested.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df.drop(columns=['PassengerId','Name','Ticket','Cabin'], inplace=True)

In [9]:
df.isna().sum()

Survived     0
Pclass       0
Sex          0
Age         86
SibSp        0
Parch        0
Fare         1
Embarked     0
dtype: int64

In [5]:
#train test split
x_train, x_test, y_train, y_test =  train_test_split(df.drop(columns=['Survived']),df['Survived'],random_state=42, test_size=0.2)

In [8]:
x_train.head(2)

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0,S
31,2,male,24.0,2,0,31.5,S


In [12]:
# imputer transformer
#using index instead of column name
trf1 = ColumnTransformer([
    ('imputed_age',SimpleImputer(strategy='mean'),[2]),
    ('imputed_embarked',SimpleImputer(strategy='most_frequent'),[6])],
     remainder='passthrough')

In [13]:
# One hot encoding transformer
#encoding sex and embarked

trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False, handle_unknown='ignore'),[1,6])
], remainder='passthrough')

In [19]:
#scaling all the columns
trf3 = ColumnTransformer([
    ('scale',MinMaxScaler(),slice(0,10))
])

In [15]:
# feature selection transformer
trf4= SelectKBest(score_func=chi2,k=8)

In [16]:
trf5=DecisionTreeClassifier()

In [20]:
#making of the pipeline

pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),

])

In [21]:
#had it been only pre processing steps we would use fit_tranform() istead of fit but since we have a model intact in the pipeline we use fit()
pipe.fit(x_train,y_train)

c:\Users\ravik\anaconda3\envs\ml_engine\Lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('imputed_age',
                                                  SimpleImputer(), [2]),
                                                 ('imputed_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x000001B6E112C680>)),
                ('trf5', DecisionTreeClassifier())])

In [22]:
y_pred=pipe.predict(x_test)

In [23]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.5952380952380952